In [1]:
import os
import time
import numpy as np
import cv2 
from matplotlib import pyplot as plt
import scipy.io as sio
import sys
import numpy as np
import platform
import statistics



def apply_FAST(source):
    #here fast is only used for detecting the keypoints but later on BRIEF will be used for the 
    img = cv2.imread(source,0)     
    # find the keypoints and descriptors with SIFT
    #kp1, des1 = orb.detectAndCompute(img1,None)
    #kp2, des2 = orb.detectAndCompute(img2,None)   
    
    # Initiate FAST extractor
    #fast = cv2.FastFeatureDetector_create()
    #kp = fast.detect(img,None)
    
    # Initiate BRIEF extractor
    #brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
    #kp1, des1 = brief.compute(img, kp)
    
    brisk = cv2.BRISK_create(thresh=10, octaves=1)
    kp1, des1 = brisk.detectAndCompute(img,None)
    #print(len(kp1))
    #print(len(des1))
    return kp1, des1




def MATCH_CUSTOM(input1, input2):
    name = input1.split("_")
    number_of_matches = 20;       
    k1, d1 = apply_FAST(input1)
    k2, d2 = apply_FAST(input2)
    #print("Feature Lengths", len(k1), len(k2))
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    # Match descriptors.
    #matches = bf.match(d1,d2)
    matches = bf.knnMatch(d1,d2, k=2)
    # Sort them in the order of their distance.
    #matches = sorted(matches, key = lambda x:x.distance)




    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.6 * n.distance:
            good.append([m])
        
    #print("Good Length: ", len(good))
    #print("For entry ", name[0],"between", input1, "and ", input2," Feature Lengths", len(k1), len(k2), "Maches : ", len(good))
    return len(good)
    #img1 = cv2.imread(input1,0)
    #img2 = cv2.imread(input2,0)
    #img3 = cv2.drawMatches(img1,k1,img2,k2, matches[:number_of_matches] ,None, flags=2)
    #img3 = cv2.drawMatchesKnn(img1,k1,img2,k2,good,None, flags=2)
    #plt.imshow(img3),plt.show()


#Function to get the list of files in a supplied directory
def files(path):  
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file


t = time.time()
            
            
            
print(sys.version)
platform.python_version()
PWD1 = "017_reference"
entries = list()
file_list = files(PWD1)
for file in file_list:
    if "PNG" in file:
        #print(file)
        entries.append(file)
print(entries)



#Training Stage
match_values = list()
number_of_entries = len(entries)
data = list()
for i in range(0, number_of_entries):
    for j in range(i+1, number_of_entries):
        #print(i, j)
        match_values.append(MATCH_CUSTOM(PWD1 + "\\" + entries[i],PWD1 + "\\" +  entries[j]))
        data.append([i,j,match_values[len(match_values)-1]])
print(match_values)
print(data)

mean_ = int(sum(match_values)/len(match_values))
std_dev = int(statistics.stdev(match_values))

diff = -1;
reference = None
index = -5
mean_new = -5
for x in range(0, len(match_values)):
    if match_values[x] > diff:
        diff = match_values[x]
        reference = entries[data[x][0]]
        index = x       
print(reference)
print(str(mean_) + "\n \n\n _______________________Training Stage ends here______________________\n\n\n")




#TestingStage
#Confustion_Matrix_base
TP_true_positive  = 0
FN_false_negative = 0
FP_false_positive = 0
TN_true_negative  = 0


PWD2 = "017_questioned"
entries = list()
file_list = files(PWD2)
for file in file_list:
    if "PNG" in file:
        #print(file)
        #actual_class_false +=1
        entries.append(file)
    if "png" in file:
        #print(file)
        #actual_class_true +=1
        entries.append(file)
#print(entries)

threshold_factor = [0, 0.5, 1, 1.5, 2, 2.5, 3];

for tf in threshold_factor:

    #Confustion_Matrix_base
    TP_true_positive  = 0
    FN_false_negative = 0
    FP_false_positive = 0
    TN_true_negative  = 0

    for x in entries:
        match_count = MATCH_CUSTOM(PWD2 + "\\" + x ,PWD1 + "\\" +  reference)
        if len(x) == 10:
            if abs(match_count-mean_) <= std_dev*tf:
                TP_true_positive += 1
            else:
                FN_false_negative += 1
        else:
            if abs(match_count-mean_) <= std_dev*tf:
                FP_false_positive +=1
            else:
                TN_true_negative +=1

    #print("TP = " + str(TP_true_positive) + " FN = " + str(FN_false_negative) )
    #print("FP = " + str(FP_false_positive) + " TN = " + str(TN_true_negative) )
    
    TPR = TP_true_positive /(TP_true_positive + FN_false_negative)
    FPR = FP_false_positive / (FP_false_positive + TN_true_negative)

    accuracy = (TP_true_positive+ TN_true_negative)/24
    precision = TP_true_positive/(TP_true_positive+FP_false_positive)
    recall = TP_true_positive / (TP_true_positive + FN_false_negative)
    f_measure = 2 / (1/precision+1/recall)

    #print("Accuracy : " + str ( accuracy))   
    #print("Precision: " + str(precision))
    #print("Recall   : " + str(recall))
    #print("F-measure: " + str(f_measure))
    
    print("True positive rate : " + str(TPR))
    print("False positive rate: " + str(FPR))

    elapsed = time.time() - t
    #print("Time elapsed : " + str(elapsed))



        
    
        




        




3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
['13_017.PNG', '14_017.PNG', '15_017.PNG', '16_017.PNG', '17_017.PNG', '18_017.PNG', '19_017.PNG', '20_017.PNG', '21_017.PNG', '22_017.PNG', '23_017.PNG', '24_017.PNG']
[4, 3, 4, 4, 9, 1, 1, 4, 3, 1, 3, 4, 4, 6, 6, 5, 2, 7, 3, 4, 3, 8, 9, 5, 5, 4, 4, 6, 6, 4, 12, 6, 8, 2, 3, 5, 8, 6, 1, 6, 6, 6, 8, 4, 5, 4, 3, 2, 2, 5, 5, 2, 4, 6, 5, 6, 1, 2, 2, 7, 3, 4, 4, 3, 9, 7]
[[0, 1, 4], [0, 2, 3], [0, 3, 4], [0, 4, 4], [0, 5, 9], [0, 6, 1], [0, 7, 1], [0, 8, 4], [0, 9, 3], [0, 10, 1], [0, 11, 3], [1, 2, 4], [1, 3, 4], [1, 4, 6], [1, 5, 6], [1, 6, 5], [1, 7, 2], [1, 8, 7], [1, 9, 3], [1, 10, 4], [1, 11, 3], [2, 3, 8], [2, 4, 9], [2, 5, 5], [2, 6, 5], [2, 7, 4], [2, 8, 4], [2, 9, 6], [2, 10, 6], [2, 11, 4], [3, 4, 12], [3, 5, 6], [3, 6, 8], [3, 7, 2], [3, 8, 3], [3, 9, 5], [3, 10, 8], [3, 11, 6], [4, 5, 1], [4, 6, 6], [4, 7, 6], [4, 8, 6], [4, 9, 8], [4, 10, 4], [4, 11, 5], [5, 6, 4], [5, 7, 3], [5, 8, 2], [5, 9, 